# Hardware Emulation and PYNQ

This notebook shows how to set up PYNQ to use the Vitis hardware emulator rather than real hardware. The starting point is to build the emulation xclbin file which follows the exact same steps as for building one for real hardware. You can see the *Building with Vitis* notebook alongside this one for more details on the commands used.

The only change we need to make is to pass `-t hw_emu` to the `v++` tool.

In [1]:
%%writefile vadd.c

void vadd(int* in_a, int* in_b, int* out_c, int count) {
#pragma HLS INTERFACE m_axi port=in_a offset=slave
#pragma HLS INTERFACE s_axilite port=in_a bundle=control
#pragma HLS INTERFACE m_axi port=in_b offset=slave
#pragma HLS INTERFACE s_axilite port=in_b bundle=control
#pragma HLS INTERFACE m_axi port=out_c offset=slave
#pragma HLS INTERFACE s_axilite port=out_c bundle=control
#pragma HLS INTERFACE s_axilite port=count bundle=control
#pragma HLS INTERFACE s_axilite port=return bundle=control
    for (int i = 0; i < count; ++i) {
        *out_c++ = *in_a++ + *in_b++;
    }
}

Overwriting vadd.c


In [2]:
import glob

platform = glob.glob("/opt/xilinx/platforms/*/*.xpfm")[0]

In [3]:
!v++ -c vadd.c -t hw_emu --kernel vadd -f $platform -o vadd.xo

Option Map File Used: '/opt/xilinx/2019.2/data/vitis/vpp/optMap.xml'

****** v++ v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ compile can be found at:
	Reports: /home/xilinx/notebook/_x/reports/vadd
	Log files: /home/xilinx/notebook/_x/logs/vadd
Running Dispatch Server on port:34805
INFO: [v++ 60-1548] Creating build summary session with primary output /home/xilinx/notebook/vadd.xo.compile_summary, at Mon Jan 27 20:04:45 2020
INFO: [v++ 60-1316] Initiating connection to rulecheck server, at Mon Jan 27 20:04:45 2020
Running Rule Check Server on port:36071
INFO: [v++ 60-1315] Creating rulecheck session with output '/home/xilinx/notebook/_x/reports/vadd/v++_compile_vadd_guidance.html', at Mon Jan 27 20:04:47 2020
INFO: [v++ 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u200_xdma_201830_2/xilinx_u200_xdma_201830_2.xpfm
INFO:

The output is the `vadd.xo` file which contains the compiled kernel

In [10]:
!v++ -l -t hw_emu -o vadd_emu.xclbin -f $platform vadd.xo

Option Map File Used: '/opt/xilinx/2019.2/data/vitis/vpp/optMap.xml'

****** v++ v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /home/xilinx/notebook/_x/reports/link
	Log files: /home/xilinx/notebook/_x/logs/link
Running Dispatch Server on port:40752
INFO: [v++ 60-1548] Creating build summary session with primary output /home/xilinx/notebook/vadd_emu.xclbin.link_summary, at Mon Jan 27 20:06:19 2020
INFO: [v++ 60-1316] Initiating connection to rulecheck server, at Mon Jan 27 20:06:19 2020
Running Rule Check Server on port:43125
INFO: [v++ 60-1315] Creating rulecheck session with output '/home/xilinx/notebook/_x/reports/link/v++_link_vadd_emu_guidance.html', at Mon Jan 27 20:06:20 2020
INFO: [v++ 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u200_xdma_201830_2/xilinx_u200_xdma_201830_2.xpfm
IN

## Setting up the emulator

Now we have a bitstream we can start setting up the emulator. First we use the `emconfigutil` utility to define the system we want to emulate. In this case we want a single instance containing the platform we used.

In [11]:
!emconfigutil -f $platform --nd 1


****** configutil v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

INFO: [ConfigUtil 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u200_xdma_201830_2/xilinx_u200_xdma_201830_2.xpfm
INFO: [ConfigUtil 60-1578]   This platform contains Device Support Archive '/opt/xilinx/platforms/xilinx_u200_xdma_201830_2/hw/xilinx_u200_xdma_201830_2.dsa'
emulation configuration file `emconfig.json` is created in current working directory 


The `emconfig.json` file describes the platform and is used by the emulator runtime. default search path for this file is alongside the executable which would be `/usr/bin/python3` for a Python script. We can override this location by setting the `EMCONFIG_PATH` environment variable.

In [12]:
import os
os.environ['EMCONFIG_PATH'] = os.environ['PWD']

We also need to set the `XCL_EMULATION_MODE` environment variable to tell PYNQ and the underlying runtime to run against the emulator rather than real hardware. Note that this must be done before importing PYNQ.

In [13]:
os.environ['XCL_EMULATION_MODE'] = 'hw_emu'

## Running PYNQ on the Emulator

In [14]:
import pynq

Now we can instantiate our overlay - this will start the emulator in the background

In [15]:
ol = pynq.Overlay('vadd_emu.xclbin')

We can now use PYNQ in exactly the same way as if we were running against real hardware. For more details on the commands used refer to the _introduction_ notebook. The only change made here is to reduce the number of elements in the array to reduce the time it takes for the emulation to run.

In [19]:
import numpy as np

vadd = ol.vadd_1

in1 = pynq.allocate((1024,), 'u4')
in2 = pynq.allocate((1024,), 'u4')
out = pynq.allocate((1024,), 'u4')

in1[:] = np.random.randint(low=0, high=100, size=(1024,), dtype='u4')
in2[:] = 200

in1.sync_to_device()
in2.sync_to_device()

vadd.call(in1, in2, out, 1024)

out.sync_from_device()

out

PynqBuffer([214, 277, 241, ..., 213, 208, 237], dtype=uint32)

The result looks promising but we can double check with numpy

In [20]:
np.array_equal(out, in1 + in2)

True

Using timeit we can see just how much slower the hardware emulation is compared to the software implementation.

In [23]:
%%timeit
in1.sync_to_device()
in2.sync_to_device()

vadd.call(in1, in2, out, 1024)

out.sync_from_device()

5.56 s ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


When using emulation it's important to close the device as well as free the bitstream to make the sure the simulator running the background is correctly shut down

In [24]:
ol.free()
pynq.Device.active_device.close()

For more information on using the simulator refer the [Vitis debugging documentation](https://www.xilinx.com/html_docs/xilinx2019_2/vitis_doc/Chunk1273465831.html#pht1538574247013). In particular this will show how to

 * View waveforms of the accelerator execution
 * Debug the underlying hardware code interactively
 
Copyright (C) 2020 Xilinx, Inc